In [1]:
import numpy as np
import os
import pandas as pd
import run_openpose
import re
import cv2
import matplotlib.pyplot as plt

In [2]:
!ls
img_show_flag = False

Photos                    depth_test_notebook.ipynb run_openpose.py
README.md                 openpose_depth_test.py
__pycache__               pose


In [3]:
colors = [ [0,100,255], [0,100,255], [0,255,255], [0,100,255], [0,255,255], [0,100,255],
         [0,255,0], [255,200,100], [255,0,255], [0,255,0], [255,200,100], [255,0,255],
         [0,0,255], [255,0,0], [200,200,0], [255,0,0], [200,200,0], [0,0,0]]

In [4]:
directory = os.path.join('.','photos')
filename = os.path.join(directory,'annotations.json')

In [5]:
data=pd.read_json(filename)

In [6]:
data = data.transpose().reset_index()[['filename','regions','size']]

In [7]:
protoFile = os.path.join('.','pose','coco','pose_deploy_linevec.prototxt')
weightsFile = os.path.join('.','pose','coco','pose_iter_440000.caffemodel')

In [8]:
dist_data=[]

In [ ]:
for filename in os.listdir(directory):
    if filename.endswith('.jpg'):
        dist={'Filename': filename, 'Distance': int(re.search('%s(.*)%s' % ('_', 'm'), filename).group(1))}
        print(os.path.join(directory, filename))
        estimate = run_openpose.run_openpose(os.path.join(directory, filename),protoFile,weightsFile)
        f=data['filename']==filename
        d=data['regions'][[i for i, x in enumerate(f) if x][0]]
        min_y = 10000000000000
        max_y = -1
        for e_point in estimate:
            if e_point:
                idx = e_point[0][3]
                e_x = e_point[0][0]
                e_y = e_point[0][1]
                if (idx > 0) & (idx < 14):
                    if d[idx]['shape_attributes']:
                        x = d[idx]['shape_attributes']['cx']
                        y = d[idx]['shape_attributes']['cy']
                        min_y = min([y, min_y])
                        max_y = max([y, max_y])
                        dist[idx] = np.sqrt((e_x-x)**2+(e_y-y)**2)
        dist['Person Height'] = abs(max_y-min_y)
        if img_show_flag:
            frameClone = cv2.imread(os.path.join(directory, filename))
            for e_point in estimate[1:14]:
                cv2.circle(frameClone, tuple(e_point[0][0:2]), 5, colors[e_point[0][3]], -1)
                aux = d[e_point[0][3]]['shape_attributes']
                cv2.circle(frameClone, (aux['cx'], aux['cy']), 5, colors[e_point[0][3]//10], -1)
            cv2.imshow("Keypoints",frameClone)
            cv2.waitKey(0)

        dist_data.append(dist)

./photos/D_3m_2.jpg
./photos/D_3m_1.jpg
./photos/M_4m_2.jpg
./photos/D_7m_2.jpg
./photos/M_4m_1.jpg
./photos/D_15m_1.jpg
./photos/D_7m_1.jpg


In [ ]:
dist_dataframe = pd.DataFrame(dist_data) 
dist_dataframe.sort_values(by=['Distance'])

In [ ]:
aux=np.array(dist_dataframe['Distance'])

In [ ]:
distances = np.unique(aux)
mean_dist = 100000*np.ones(distances.shape)

for j,i in enumerate(distances):
    temp = np.array(dist_dataframe[dist_dataframe.columns[2:]][dist_dataframe['Distance']==i].div(dist_dataframe[dist_dataframe.columns[-1]][dist_dataframe['Distance']==i]))
    mean_dist[j] = np.nanmean(temp)
    
#old
#distances = np.unique(aux)
#mean_dist = 100000*np.ones(distances.shape)

#for j,i in enumerate(distances):
#    temp = np.array(dist_dataframe[dist_dataframe.columns[2:]][dist_dataframe['Distance']==i])
#    mean_dist[j] = np.nanmean(temp)
    
  

In [ ]:
dist_dataframe[dist_dataframe.columns[2:]][dist_dataframe['Distance']==i]

In [ ]:
%matplotlib inline

index = np.arange(len(distances))
plt.bar(index, mean_dist)
plt.xlabel('Distances(m)', fontsize=10)
plt.ylabel('Euclidean dist. normalized w.r.t. target height', fontsize=10)
plt.xticks(index, distances, fontsize=10, rotation=0)
plt.title('Keypoint detection with Distance')
plt.show()

print(mean_dist)